In [1]:
import numpy as np
import random
import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from IPython.display import Image
from transvae import trans_models
from transvae.transformer_models import TransVAE
from transvae.rnn_models import RNN, RNNAttn
from transvae.wae_models import WAE
from transvae.aae_models import AAE
from transvae.tvae_util import *
from transvae import analysis
import glob
import re

from sklearn.decomposition import PCA
from sklearn.manifold import Isomap
from sklearn import metrics
from sklearn.manifold import trustworthiness
from scipy.stats import pearsonr
from scipy.stats import spearmanr
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import NullFormatter
import plotly.express as px
import plotly.graph_objects as go

import coranking #coranking.readthedocs.io
from coranking.metrics import trustworthiness, continuity, LCMC
from transvae.snc import SNC #github.com/hj-n/steadiness-cohesiveness

def loss_plots(loss_src):
    tot_loss = analysis.plot_loss_by_type(src,loss_types=['tot_loss'])
    plt.savefig(save_dir+'tot_loss.png')
    recon_loss = analysis.plot_loss_by_type(src,loss_types=['recon_loss'])
    plt.savefig(save_dir+'recon_loss.png')
    kld_loss = analysis.plot_loss_by_type(src,loss_types=['kld_loss'])
    plt.savefig(save_dir+'kld_loss.png')
    prob_bce_loss = analysis.plot_loss_by_type(src,loss_types=['prop_bce_loss'])
    plt.savefig(save_dir+'prob_bce_loss.png')
    if 'aae' in src:
        disc_loss = analysis.plot_loss_by_type(src,loss_types=['disc_loss'])
        plt.savefig(save_dir+'disc_loss.png')
    if 'wae' in src:
        mmd_loss = analysis.plot_loss_by_type(src,loss_types=['mmd_loss'])
        plt.savefig(save_dir+'mmd_loss.png')
    plt.close('all')
    
def load_reconstructions(data,data_1D,latent_size, load_src, true_props=None,subset=None):
    
    recon_src = load_src+model.name+"_"+re.split('(\d{2,3})',latent_size[0])[0]+"_"+re.split('(\d{2,3})',latent_size[0])[1]+"//saved_info.csv"
    recon_df = pd.read_csv(recon_src)
    reconstructed_seq = recon_df['reconstructions'].to_list()[:num_sequences]
    props = torch.Tensor(recon_df['predicted properties'][:num_sequences])
    true_props_data = pd.read_csv(true_props).to_numpy()
    true_props = true_props_data[0:num_sequences,0]
    
    if subset:
        testing = pd.read_csv(subset).to_numpy()
        test_idx_list = [np.where(data==testing[idx][0]) for idx in range(len(testing))]


        batch_recon_len = len(reconstructed_seq)
        reconstructed_seq = [reconstructed_seq[test_idx_list[i][0][0]] for i in range(len(test_idx_list)) if test_idx_list[i][0][0]<batch_recon_len]
        data_1D= [data_1D[test_idx_list[i][0][0]] for i in range(len(test_idx_list)) if test_idx_list[i][0][0]<batch_recon_len]
        props = [props[test_idx_list[i][0][0]] for i in range(len(test_idx_list)) if test_idx_list[i][0][0]<batch_recon_len]
        props=torch.Tensor(props)
        data = testing[:][0]
        true_props_data = pd.read_csv(true_props).to_numpy()
        true_props = true_props_data[0:num_sequences,0]
        true_props= [true_props[test_idx_list[i][0][0]] for i in range(len(test_idx_list)) if test_idx_list[i][0][0]<batch_recon_len]

    return data, data_1D, true_props, props, reconstructed_seq

########################################################################################
gpu = True

num_sequences = 500#_000
batch_size = 200 #setting for reconstruction
example_data = 'data\\peptides\\datasets\\uniprot_v2\\peptide_train.txt'
save_dir_loc = 'model_analyses\\train\\' #folder in which to save outpts
save_dir_name = 'train' #appended to identify data: train|test|other|etc...

reconstruct=True #True:reconstruct data here; False:load reconstructions from file
recon_src = "checkpointz//analyses_ckpts//" #directory in which all reconstructions are stored
true_prop_src = 'data\\peptides\\datasets\\uniprot_v2\\function_train.txt' #if property predictor load the true labels
subset_src = "" #(optional) this file should have the true sequences for a subset of the "example data" above

ckpt_list = glob.glob(""+"checkpointz\\to_slurm//**//*.ckpt", recursive = True) #grab all checkpoint
print('current working directory: ',os.getcwd())


for i in range(len(ckpt_list)):
    #search the current directory for the model name and load that model
    model_dic = {'trans':'TransVAE','aae':'AAE','rnnattn':'RNNAttn','rnn':'RNN','wae':'WAE'}
    model_src = ckpt_list[i]
    print('working on: ',model_src,'\n')
    model_name = list(filter(None,[key for key in model_dic.keys() if key in model_src.split('\\')[-1]]))
    model = locals()[model_dic[model_name[0]]](load_fn=model_src) #use locals to call model specific constructor
    
    #create save directory for the current model according to latent space size
    latent_size = re.findall('(latent[\d]{2,3})', model_src)
    save_dir= save_dir_loc+model.name+"_"+latent_size[0]+"_"+save_dir_name
    if not os.path.exists(save_dir):os.mkdir(save_dir) 
    save_dir= save_dir+"//" 
    save_df = pd.DataFrame() #this will hold the number variables and save to CSV
    
    #load the true labels
    data = pd.read_csv(example_data).to_numpy() 
    data_1D = data[:num_sequences,0] #gets rid of extra dimension
    true_props_data = pd.read_csv(true_prop_src).to_numpy()
    true_props = true_props_data[0:num_sequences,0]

    
    #get the log.txt file from the ckpt and model name then plot loss curves
    loss_src = '_'.join( ("log",model_src.split('\\')[-1].split('_')[1],model_src.split('\\')[-1].split('_')[2][:-4]+"txt") )
    src= '\\'.join([str(i) for i in model_src.split('\\')[:-1]])+"\\"+loss_src
    print(loss_src, src)
    loss_plots(src)
    
#     #set the batch size and reconstruct the data
#     model.params['BATCH_SIZE'] = batch_size
#     if reconstruct:
#         reconstructed_seq, props = model.reconstruct(data[:num_sequences], log=False, return_mems=False)
#     else:
#         data, data_1D, true_props, props, reconstructed_seq = load_reconstructions(data, data_1D,latent_size,
#                                                                                    load_src=recon_src,
#                                                                                    true_props=true_prop_src)
#     if gpu:torch.cuda.empty_cache() #free allocated CUDA memory
    
#     #save the metrics to the dataframe
#     save_df['reconstructions'] = reconstructed_seq #placing the saves on a line separate from the ops allows for editing
#     save_df['predicted properties'] = [prop.item() for prop in props[:len(reconstructed_seq)]]
#     prop_acc, prop_conf, MCC=calc_property_accuracies(props[:len(reconstructed_seq)],true_props[:len(reconstructed_seq)], MCC=True)
#     save_df['property prediction accuracy'] = prop_acc
#     save_df['property prediction confidence'] = prop_conf
#     save_df['MCC'] = MCC
    

# #   First we tokenize the input and reconstructed smiles
#     input_sequences = []
#     for seq in data_1D:
#         input_sequences.append(peptide_tokenizer(seq))
#     output_sequences = []
#     for seq in reconstructed_seq:
#         output_sequences.append(peptide_tokenizer(seq))
    
#     seq_accs, tok_accs, pos_accs, seq_conf, tok_conf, pos_conf = calc_reconstruction_accuracies(input_sequences, output_sequences)
#     save_df['sequence accuracy'] = seq_accs
#     save_df['sequence confidence'] = seq_conf
#     save_df['token accuracy'] = tok_accs
#     save_df['token confidence'] = tok_conf
#     save_df = pd.concat([pd.DataFrame({'position_accs':pos_accs,'position_confidence':pos_conf }), save_df], axis=1)
    
    ##moving into memory and entropy
    if model.model_type =='aae':
        mus, _, _ = model.calc_mems(data[:50_000], log=False, save=False) #50_000
    elif model.model_type == 'wae':
        mus, _, _ = model.calc_mems(data[:50_000], log=False, save=False) 
    else:
        mems, mus, logvars = model.calc_mems(data[:50_000], log=False, save=False) #subset size 1200*35=42000 would be ok

#     ##calculate the entropies
#     vae_entropy_mus = calc_entropy(mus)
#     save_df = pd.concat([save_df,pd.DataFrame({'mu_entropies':vae_entropy_mus})], axis=1)
#     if model.model_type != 'wae' and model.model_type!= 'aae': #these don't have a variational type bottleneck
#         vae_entropy_mems  = calc_entropy(mems)
#         save_df = pd.concat([save_df,pd.DataFrame({'mem_entropies':vae_entropy_mems})], axis=1)
#         vae_entropy_logvars = calc_entropy(logvars)
#         save_df = pd.concat([save_df,pd.DataFrame({'logvar_entropies':vae_entropy_logvars})], axis=1)
    


    #create random index and re-index ordered memory list creating n random sub-lists (ideally resulting in IID random lists)
    random_idx = np.random.permutation(np.arange(stop=mus.shape[0]))
    mus = mus[random_idx]
    shuf_data = data[random_idx]

    subsample_start=0
    subsample_length=mus.shape[0] #this may change depending on batch size

#     #(for length based coloring): record all peptide lengths iterating through input
#     pep_lengths = []
#     for idx, pep in enumerate(shuf_data[subsample_start:(subsample_start+subsample_length)]):
#         pep_lengths.append( len(pep[0]) )   
    #(for function based coloring): pull function from csv with peptide functions
    s_to_f =pd.read_csv(true_prop_src)    
    function = s_to_f['peptides'][subsample_start:(subsample_start+subsample_length)]
    function = function[random_idx] #account for random permutation

    pca = PCA(n_components=5)
    pca_batch =pca.fit_transform(X=mus[:])

#     #plot format dictionnaries
    titles={'text':'{}'.format(model.model_type.replace("_"," ").upper()),
                          'x':0.5,'xanchor':'center','yanchor':'top','font_size':40}
    general_fonts={'family':"Helvetica",'size':30,'color':"Black"}
    colorbar_fmt={'title_font_size':30,'thickness':15,'ticks':'','title_text':'Lengths',
                               'ticklabelposition':"outside bottom"}
    
#     fig = px.scatter(pd.DataFrame({"PC1":pca_batch[:,0],"PC2":pca_batch[:,1], "lengths":pep_lengths}),
#                 symbol_sequence=['hexagon2'],x='PC1', y='PC2', color="lengths",
#                 color_continuous_scale='Jet',template='simple_white', opacity=0.9)
#     fig.update_traces(marker=dict(size=9))
#     fig.update_layout(title=titles,xaxis_title="PC1", yaxis_title="PC2",font=general_fonts)
#     fig.update_coloraxes(colorbar=colorbar_fmt)
#     fig.write_image(save_dir+'pca_length.png', width=900, height=600)

#     fig = px.scatter(pd.DataFrame({"PC1":pca_batch[:,0],"PC2":pca_batch[:,1], 
#                                     "Function":list(map(lambda itm: "AMP" if itm==1 else "NON-AMP",function))}),
#                                     x='PC1', y='PC2', color="Function",symbol_sequence=['x-thin-open','circle'],
#                                     template='simple_white',symbol='Function', opacity=0.8) 
#     fig.update_traces(marker=dict(size=9))
#     fig.update_layout(title=titles,xaxis_title="PC1",yaxis_title="PC2",font=general_fonts)
#     fig.write_image(save_dir+'pca_function.png', width=900, height=600)
    
#     # Plot the explained variances
#     plt.bar(range(pca.n_components_), pca.explained_variance_ratio_*100, color='black')
#     plt.xlabel('PCA features')
#     plt.ylabel('variance %')
#     plt.xticks(range(pca.n_components_))
#     plt.savefig(save_dir+'variance_explained.png')

#     fig = px.scatter_matrix(pd.DataFrame({"PC1":pca_batch[:,0],"PC2":pca_batch[:,1],"PC3":pca_batch[:,2],
#                                           "PC4":pca_batch[:,3],"PC5":pca_batch[:,4],"lengths":pep_lengths}),
#                                     dimensions=["PC1","PC2","PC3","PC4","PC5"],
#                                     symbol_sequence=['hexagon2'],template='simple_white',
#                                     color="lengths",color_continuous_scale='Jet', opacity=0.9)
#     fig.update_traces(diagonal_visible=False)
#     fig.update_layout(title=titles,font=general_fonts)
#     fig.write_image(save_dir+'pca_matrix_length.png', width=5_000, height=2500) 
    
#     fig = px.scatter_matrix(pd.DataFrame({"PC1":pca_batch[:,0],"PC2":pca_batch[:,1],"PC3":pca_batch[:,2],
#                                           "PC4":pca_batch[:,3],"PC5":pca_batch[:,4],
#                                    "Function":list(map(lambda itm: "AMP" if itm==1 else "NON-AMP",function))}),
#                                     dimensions=["PC1","PC2","PC3","PC4","PC5"],template='simple_white',
#                                     color="Function",symbol_sequence=['x-thin','circle'],
#                                     symbol='Function', opacity=0.8) 
#     fig.update_traces(diagonal_visible=False)
#     fig.update_layout(title=titles,font=general_fonts)
#     fig.write_image(save_dir+'pca_matrix_function.png', width=5_000, height=2500) 
    pearson = {} #dict to store the pearson coefficient between PCA vs AMP function or physicochem.props.
    pearson.update({'amp'+'_spearmanr':[(spearmanr(pca_batch[:,pc],function).correlation,
                                         spearmanr(pca_batch[:,pc],function).pvalue) for pc in range(5)]})
    if 'train' in save_dir_name:
        phys_props = pd.read_csv('data\\train_physicochem_props.csv')
    else:
        phys_props = pd.read_csv('data\\test_physicochem_props.csv')

    
    for col in phys_props.columns:
        functions = phys_props[col][:len(mus)].values
        functions = functions[random_idx] #keeping track of data scrambling...
        pearson.update({str(col)+'_pearsonr':[pearsonr(pca_batch[:,pc],functions) for pc in range(5)]})
        fig = px.scatter_matrix(pd.DataFrame({"PC1":pca_batch[:,0],"PC2":pca_batch[:,1],"PC3":pca_batch[:,2],
                                                  "PC4":pca_batch[:,3],"PC5":pca_batch[:,4],
                                           "Function":functions}),
                                            dimensions=["PC1","PC2","PC3","PC4","PC5"],template='simple_white',
                                            color="Function",opacity=0.9) 
        colorbar_fmt={'title_font_size':30,'thickness':15,'ticks':'','title_text':str(col),
                               'ticklabelposition':"outside bottom"}
        fig.update_traces(diagonal_visible=False)
        fig.update_layout(title=titles,font=general_fonts)
        fig.update_coloraxes(colorbar=colorbar_fmt, 
                             cmax=np.mean(functions)+np.std(functions),
                             cmin=np.mean(functions)-np.std(functions),
                             cmid=np.mean(functions))
        fig.write_image(save_dir+col+'_PCA_matrix'+'.png', width=5_000, height=2500) 
    df_pearson = pd.DataFrame.from_dict(pearson)
    df_pearson.to_csv(save_dir+'pearsonr.csv', index=False)

#     #first calculate silhouette score on all latent space dims
#     n=15
#     latent_mem_func_subsamples = []
#     for s in range(n):
#         s_len = len(mus)//n #sample lengths
#         mem_func_sil = metrics.silhouette_score(mus[s_len*s:s_len*(s+1)], function[s_len*s:s_len*(s+1)], metric='euclidean')
#         latent_mem_func_subsamples.append(mem_func_sil)
#     save_df = pd.concat([save_df,pd.DataFrame({'latent_mem_func_silhouette':latent_mem_func_subsamples})], axis=1)
#     #then go over pairs of PC's from PCA and find max SS PC's
#     pc_pairs = [[0,1],[0,2],[0,3],[0,4],[1,2],[1,3],[1,4],[2,3],[2,4],[3,4]]
#     for pc_pair in pc_pairs:
#         print("working on PC[{},{}]".format(pc_pair[0],pc_pair[1]))
#         pca_func_subsamples = []
#         for s in range(n):
#             s_len = len(mus)//n #sample lengths
#             XY = [i for i in zip(pca_batch[s_len*s:s_len*(s+1),pc_pair[0]], pca_batch[s_len*s:s_len*(s+1),pc_pair[1]])]
#             pca_func_sil = metrics.silhouette_score(XY, function[s_len*s:s_len*(s+1)], metric='euclidean')
#             pca_func_subsamples.append(pca_func_sil)
#         save_df = pd.concat([save_df,pd.DataFrame({'pca_func_silhouette[{},{}]'.format(pc_pair[0],pc_pair[1]):pca_func_subsamples})], axis=1)
#     print( np.argmax(save_df.drop(columns=save_df.columns[0]).mean(axis=0)) )
#     save_df.to_csv(save_dir+"saved_info.csv", index=False)
    
    
    
#New section dealing with sequence generation metrics and bootstrapping from the latent space
    #first randomly sample points within the latents space
    rnd_seq_count =1_000 
    rnd_latent_list=[] #generate N latent space vectors
    mem_min = np.min(mus)
    mem_max = np.max(mus)
    for seq in range(rnd_seq_count):
        rnd_latent_list.append( np.array([random.uniform(mem_min,mem_max) for i in range(model.params['d_latent'])]).astype(np.float32) )
    
    model.params['BATCH_SIZE'] = 25
    rnd_token_list=np.empty((rnd_seq_count,model.tgt_len)) #store N decoded latent vectors now in token(0-20) form max length 125
    
    #decode these points into predicted amino acid tokens (integers)
    for batch in range(0,rnd_seq_count,model.params['BATCH_SIZE']):
        rnd_token_list[batch:batch+model.params['BATCH_SIZE']] =  model.greedy_decode(torch.tensor(rnd_latent_list[batch:batch+model.params['BATCH_SIZE']]).cuda()).cpu()
    
    #turn the tokens into characters
    decoded_rnd_seqs = decode_mols(torch.tensor(rnd_token_list), model.params['ORG_DICT'])
    decoded_rnd_seqs[:]=[x for x in decoded_rnd_seqs if x] #removes the empty lists
    
    df_gen_scores = {} #dictionnary to store results
    #UNIQUENESS
    percent_unique, unique_conf = uniqueness(decoded_rnd_seqs)
    df_gen_scores.update({'percent_unique': percent_unique})
    df_gen_scores.update({'unique_confidence':unique_conf})
    
    #NOVELTY
    #sample N test/train set sequences randomly and compare to those created
    percent_novel, novel_conf = novelty(data, np.expand_dims(np.array(decoded_rnd_seqs),1))
    df_gen_scores.update({'percent_novel':percent_novel})
    df_gen_scores.update({'novel_confidence':novel_conf})
    
    #AMP SAMPLING
    peptides_to_probe=10
    sample_count=30
    best_pc = np.argmax([np.abs(pearsonr(pca_batch[:,pc],function)[0]) for pc in range(5)]) #find the best PCvsAMP correlation
    pca_min = np.min(pca_batch[:,best_pc])
    pca_max = np.max(pca_batch[:,best_pc])
    pca_scan = np.zeros((peptides_to_probe,5)) #create a reduced vector to be sent backwards to high-D
    pca_scan[:,best_pc]=np.linspace(start=pca_min, stop=pca_max, num=peptides_to_probe) #scan 1 dim evenly with best PC
    amp_sample_latents = pca.inverse_transform(pca_scan) #inverse to high-Dims for decoding
    all_gen_seqs = [] #stored in a text file for AMP prediction later
    for idx,amp in enumerate(amp_sample_latents):
        print("working on amp sample number: ",idx)
        mus=np.expand_dims(amp.astype(np.float32),0)
        nearby_samples = np.random.normal(loc=0,scale=1,size=(sample_count,1,model.params['d_latent'])).astype(np.float32)*0.3 + mus
        model.params['BATCH_SIZE'] = 25
        rnd_token_list=np.empty((sample_count,model.tgt_len)) #store N decoded latent vectors now in token(0-20) form max length 125
        for batch in range(0,sample_count,model.params['BATCH_SIZE']):
            rnd_token_list[batch:batch+model.params['BATCH_SIZE']] =  model.greedy_decode(torch.tensor(nearby_samples[batch:batch+model.params['BATCH_SIZE']]).squeeze().cuda()).cpu()
        decoded_rnd_seqs = decode_mols(torch.tensor(rnd_token_list), model.params['ORG_DICT'])
        
               
        for seq in decoded_rnd_seqs:
            all_gen_seqs.append(seq)
        decoded_rnd_seqs = [seq for seq in decoded_rnd_seqs if len(seq)>0] #drop the empy sequences
        
        #SEQ SIMILARITY
        similarity_score = sequence_similarity(decoded_rnd_seqs)  
        df_gen_scores.update({'average_sequence_similarity_'+str(idx): np.average(similarity_score)})
        df_gen_scores.update({'std_on_similarity_score_'+str(idx): np.std(similarity_score)})
        
        #AMP UNIQUENESS
        amp_percent_unique, amp_unique_conf = uniqueness(decoded_rnd_seqs)
        df_gen_scores.update({'amp_uniqueness_'+str(idx): amp_percent_unique})
        df_gen_scores.update({'amp_uniqueness_std_'+str(idx): amp_unique_conf})
        
        #Jaccard Similarity Score
        jac_scores_2 = jaccard_similarity_score(decoded_rnd_seqs,2)
        jac_scores_3 = jaccard_similarity_score(decoded_rnd_seqs,3)
        df_gen_scores.update({'amp_jac_score_2_'+str(idx): np.average(jac_scores_2)})
        df_gen_scores.update({'amp_jac_score_std_2_'+str(idx): np.std(jac_scores_2)})
        df_gen_scores.update({'amp_jac_score_3_'+str(idx): np.average(jac_scores_3)})
        df_gen_scores.update({'amp_jac_score_std_3_'+str(idx): np.std(jac_scores_3)})
    
    #Store Output
    with open(save_dir+'all_gen_seqs.txt','w') as f:
        for seq in all_gen_seqs:
            f.write(str(seq)+"\n")
    df = pd.DataFrame.from_dict([df_gen_scores])
    pd.DataFrame.from_dict([df_gen_scores]).to_csv(save_dir+"generation_metrics.csv", index=False)
    

current working directory:  C:\Users\s_renaud\Documents\GitHub\MSCSAM_TBD\main_model
working on:  checkpointz\to_slurm\aae_latent128\300_aae-128_peptide.ckpt 

log_aae-128_peptide.txt checkpointz\to_slurm\aae_latent128\log_aae-128_peptide.txt
aae-128_peptide


C:\Users\s_renaud\AppData\Local\Temp/ipykernel_14252/1293800399.py:312: UserWarning:

Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ..\torch\csrc\utils\tensor_new.cpp:201.)



decoding sequences of max length  125 current position:  0
decoding sequences of max length  125 current position:  100
decoding sequences of max length  125 current position:  0
decoding sequences of max length  125 current position:  100
decoding sequences of max length  125 current position:  0
decoding sequences of max length  125 current position:  100
decoding sequences of max length  125 current position:  0
decoding sequences of max length  125 current position:  100
decoding sequences of max length  125 current position:  0
decoding sequences of max length  125 current position:  100
decoding sequences of max length  125 current position:  0
decoding sequences of max length  125 current position:  100
decoding sequences of max length  125 current position:  0
decoding sequences of max length  125 current position:  100
decoding sequences of max length  125 current position:  0
decoding sequences of max length  125 current position:  100
decoding sequences of max length  125 cu

C:\Users\s_renaud\Documents\GitHub\MSCSAM_TBD\main_model\transvae\trans_models.py:54: UserWarning:

To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).



log_aae-128_peptide.txt checkpointz\to_slurm\aae_latent32\log_aae-128_peptide.txt
aae-128_peptide
decoding sequences of max length  125 current position:  0
decoding sequences of max length  125 current position:  100
decoding sequences of max length  125 current position:  0
decoding sequences of max length  125 current position:  100
decoding sequences of max length  125 current position:  0
decoding sequences of max length  125 current position:  100
decoding sequences of max length  125 current position:  0
decoding sequences of max length  125 current position:  100
decoding sequences of max length  125 current position:  0
decoding sequences of max length  125 current position:  100
decoding sequences of max length  125 current position:  0
decoding sequences of max length  125 current position:  100
decoding sequences of max length  125 current position:  0
decoding sequences of max length  125 current position:  100
decoding sequences of max length  125 current position:  0
dec

...

decoding sequences of max length  125 current position:  0
decoding sequences of max length  125 current position:  100
decoding sequences of max length  125 current position:  0
decoding sequences of max length  125 current position:  100
decoding sequences of max length  125 current position:  0
decoding sequences of max length  125 current position:  100
decoding sequences of max length  125 current position:  0
decoding sequences of max length  125 current position:  100
decoding sequences of max length  125 current position:  0
decoding sequences of max length  125 current position:  100
decoding sequences of max length  125 current position:  0
decoding sequences of max length  125 current position:  100
decoding sequences of max length  125 current position:  0
decoding sequences of max length  125 current position:  100
decoding sequences of max length  125 current position:  0
decoding sequences of max length  125 current position:  100
decoding sequences of max length  125 cu

C:\Users\s_renaud\Documents\GitHub\MSCSAM_TBD\main_model\transvae\trans_models.py:54: UserWarning:

To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).



log_wae-128_peptide.txt checkpointz\to_slurm\wae_latent32\log_wae-128_peptide.txt
wae-128_peptide
decoding sequences of max length  125 current position:  0
decoding sequences of max length  125 current position:  100
decoding sequences of max length  125 current position:  0
decoding sequences of max length  125 current position:  100
decoding sequences of max length  125 current position:  0
decoding sequences of max length  125 current position:  100
decoding sequences of max length  125 current position:  0
decoding sequences of max length  125 current position:  100
decoding sequences of max length  125 current position:  0
decoding sequences of max length  125 current position:  100
decoding sequences of max length  125 current position:  0
decoding sequences of max length  125 current position:  100
decoding sequences of max length  125 current position:  0
decoding sequences of max length  125 current position:  100
decoding sequences of max length  125 current position:  0
dec

C:\Users\s_renaud\Documents\GitHub\MSCSAM_TBD\main_model\transvae\trans_models.py:54: UserWarning:

To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).



log_wae-128_peptide.txt checkpointz\to_slurm\wae_latent64\log_wae-128_peptide.txt
wae-128_peptide
decoding sequences of max length  125 current position:  0
decoding sequences of max length  125 current position:  100
decoding sequences of max length  125 current position:  0
decoding sequences of max length  125 current position:  100
decoding sequences of max length  125 current position:  0
decoding sequences of max length  125 current position:  100
decoding sequences of max length  125 current position:  0
decoding sequences of max length  125 current position:  100
decoding sequences of max length  125 current position:  0
decoding sequences of max length  125 current position:  100
decoding sequences of max length  125 current position:  0
decoding sequences of max length  125 current position:  100
decoding sequences of max length  125 current position:  0
decoding sequences of max length  125 current position:  100
decoding sequences of max length  125 current position:  0
dec

<H4>Since Compute Canada does not do the dimensionality reduction metrics we need to do them below

In [1]:
import coranking #coranking.readthedocs.io
from coranking.metrics import trustworthiness, continuity, LCMC
from transvae.snc import SNC #github.com/hj-n/steadiness-cohesiveness

import numpy as np
import os
import pandas as pd
import seaborn as sns
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from IPython.display import Image
from transvae import trans_models
from transvae.transformer_models import TransVAE
from transvae.rnn_models import RNN, RNNAttn
from transvae.wae_models import WAE
from transvae.aae_models import AAE
from transvae.tvae_util import *
from transvae import analysis
import glob
import re


gpu = True

example_data = 'data\\peptides\\datasets\\uniprot_v2\\peptide_test.txt'
test_train='test'
ckpt_list = glob.glob(""+"checkpointz\\to_slurm//**//*.ckpt", recursive = True) #grab all checkpoints
analyses_list = glob.glob("model_analyses\\test//**/*.csv", recursive=True) #grab all analyses
print('current working directory: ',os.getcwd())

for i in range(len(ckpt_list)):
    
    #search the current directory for the model name and load that model
    model_dic = {'trans':'TransVAE','aae':'AAE','rnnattn':'RNNAttn','rnn':'RNN','wae':'WAE'}
    model_src = ckpt_list[i]
    print('working on: ',model_src,'\n')
    model_name = list(filter(None,[key for key in model_dic.keys() if key in model_src.split('//')[-1]]))
    model = locals()[model_dic[model_name[0]]](load_fn=model_src) #use locals to call model specific constructor
    
    #load the analysis file corresponding to the model from the CC outputs
    for idx in range(len(analyses_list)):
        if analyses_list[idx].split("\\")[-2].find(model_src.split("\\")[-2].split("_")[0]) != -1 and analyses_list[idx].split("\\")[-2].find(model_src.split("\\")[-2].split("_")[1]) != -1:
            if analyses_list[idx].find("rnnattn")  != -1 and model_src.find("rnnattn") == -1: continue
            save_dir = analyses_list[idx]
            cur_analysis = pd.read_csv(save_dir)
    print("analysis: ",save_dir, "checkpoint: ",model_src)
    save_df = cur_analysis #this will hold the number variables and save to CSV
    
    #load the true labels
    data = pd.read_csv(example_data).to_numpy() 
    data_1D = data[:,0] #gets rid of extra dimension
    
    #moving into memory and entropy
    if model.model_type =='aae':
        mus, _, _ = model.calc_mems(data[:60_000], log=False, save=False) 
    elif model.model_type == 'wae':
        mus, _, _ = model.calc_mems(data[:60_000], log=False, save=False) 
    else:
        mems, mus, logvars = model.calc_mems(data[:60_000], log=False, save=False) #subset size 1200*35=42000 would be ok

    #create random index and re-index ordered memory list creating n random sub-lists (ideally resulting in IID random lists)
    random_idx = np.random.permutation(np.arange(stop=mus.shape[0]))
    mus[:] = mus[random_idx]
    data = data[random_idx]
    
    #need to perform PCA to be able to compare dimensionality reduction quality
    pca = PCA(n_components=2)
    pca_batch =pca.fit_transform(X=mus[:])
    
    #now ready to calculation dimensionality reduction accuracy with metrics
    trust_subsamples = []
    cont_subsamples = []
    lcmc_subsamples = []
    steadiness_subsamples = []
    cohesiveness_subsamples = []
    if 'test' in test_train: #different number of bootsraps for train vs test
        n=15
    else:
        n=15
    parameter = { "k": 50,"alpha": 0.1 } #for steadiness and cohesiveness
    for s in range(n):
        s_len = len(mus)//n
        Q = coranking.coranking_matrix(mus[s_len*s:s_len*(s+1)], pca_batch[s_len*s:s_len*(s+1)])
        trust_subsamples.append( np.mean(trustworthiness(Q, min_k=1, max_k=50)) )
        cont_subsamples.append( np.mean(continuity(Q, min_k=1, max_k=50)) )
        lcmc_subsamples.append( np.mean(LCMC(Q, min_k=1, max_k=50)) )
        print(s,trust_subsamples[s],cont_subsamples[s],lcmc_subsamples[s])

        metrics = SNC(raw=mus[s_len*s:s_len*(s+1)], emb=pca_batch[s_len*s:s_len*(s+1)], iteration=300, dist_parameter=parameter)
        metrics.fit() #solve for steadiness and cohesiveness
        steadiness_subsamples.append(metrics.steadiness())
        cohesiveness_subsamples.append(metrics.cohesiveness())
        print(metrics.steadiness(),metrics.cohesiveness())
        Q=0 #trying to free RAM
        metrics=0
        torch.cuda.empty_cache() #free allocated CUDA memory

    save_df = pd.concat([save_df,pd.DataFrame({'latent_to_PCA_trustworthiness':trust_subsamples})], axis=1)
    save_df = pd.concat([save_df,pd.DataFrame({'latent_to_PCA_continuity':cont_subsamples})], axis=1)
    save_df = pd.concat([save_df,pd.DataFrame({'latent_to_PCA_lcmc':lcmc_subsamples})], axis=1)
    save_df = pd.concat([save_df,pd.DataFrame({'latent_to_PCA_steadiness':steadiness_subsamples})], axis=1)
    save_df = pd.concat([save_df,pd.DataFrame({'latent_to_PCA_cohesiveness':cohesiveness_subsamples})], axis=1)  
    
    save_df.to_csv(save_dir, index=False)

current working directory:  C:\Users\s_renaud\Documents\GitHub\MSCSAM_TBD\main_model
working on:  checkpointz\to_slurm\aae_latent128\300_aae-128_peptide.ckpt 

analysis:  model_analyses\test\aae-128_peptide_latent128_test\saved_info.csv checkpoint:  checkpointz\to_slurm\aae_latent128\300_aae-128_peptide.ckpt
aae-128_peptide
0 0.8302777117363083 0.8802514029830146 0.12416861109784344
0.8021271832930836 0.7131013082619109
1 0.8330010491048477 0.8849852409881493 0.11999433373022127
0.823952404375804 0.6731206728892155
2 0.8384108356678447 0.8855374320262078 0.12188120052992525
0.8182679651467646 0.6874399748799729
3 0.8382689469103471 0.8853910884692464 0.12145384925631005
0.8026112373420478 0.7055760061581462
4 0.833973408953818 0.885358476768385 0.11840152285502772
0.8130479647235245 0.6771931272866757
5 0.8369093254879482 0.8854397636166487 0.1218365053857598
0.8202771383862371 0.6806602047004187
6 0.8290242290348175 0.8792937366674517 0.11962778946796075
0.815886347895445 0.6866688563

C:\Users\s_renaud\Documents\GitHub\MSCSAM_TBD\main_model\transvae\trans_models.py:54: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.params['CHAR_WEIGHTS'] = torch.tensor(self.params['CHAR_WEIGHTS'], dtype=torch.float)


0 0.7600417815039993 0.8549679240355698 0.09881150309882607
0.7304666898304422 0.6689619082472802
1 0.7434343648815366 0.8459461372081621 0.09157987108312216
0.7230706669975837 0.6579112932757039
2 0.7601914747145255 0.8545445051397355 0.09692768962840484
0.7378277731751821 0.666614336808776
3 0.7567649615778516 0.854208817021764 0.09682214458963377
0.7443521531288059 0.6571462301357864
4 0.7613898144176768 0.8581184972761141 0.10445522843677703
0.7322508173233998 0.6523470435142786
5 0.7548625584130721 0.8510460336132378 0.09662431785452337
0.7183852097881469 0.6753079807155002
6 0.7432269004657063 0.8466054990042239 0.09273020277063593
0.7087440549940396 0.6597335740530212
7 0.7519584242294244 0.8525189814005324 0.09905583133879528
0.7181692997314675 0.6750105927211472
8 0.752505009806922 0.8541801797626228 0.09733898851517259
0.7311067895705317 0.6759535467264579
9 0.7458245907272166 0.8453162220415275 0.09246157748718734
0.7093765983524298 0.6717671727010783
10 0.755710850475864 0.

C:\Users\s_renaud\Documents\GitHub\MSCSAM_TBD\main_model\transvae\trans_models.py:54: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.params['CHAR_WEIGHTS'] = torch.tensor(self.params['CHAR_WEIGHTS'], dtype=torch.float)


0 0.7834516967932477 0.8580754526560782 0.09678576010164755
0.7604636848812987 0.6893751414419041
1 0.7743132226807024 0.8524519343878366 0.10130143373296263
0.7472723682494425 0.6834994775434828
2 0.7841020263091508 0.8577118135509808 0.10512396340324277
0.7706715304139302 0.6698788893209997
3 0.7908943461440002 0.8624430726843788 0.10725714387664365
0.7649903603736826 0.6782179627092231
4 0.7880292285189355 0.8600867235522627 0.10560710067664024
0.7625973400021329 0.6928882886782847
5 0.7757735855748614 0.8503557228141815 0.10004842355180546
0.7631725865176507 0.6806207875988295
6 0.7875464541918201 0.8580190664649961 0.10072182922849626
0.782351839716679 0.6496939456383919
7 0.7900962391566115 0.8636384393387027 0.10649006387757885
0.7812796440377164 0.6693866959580221
8 0.7855739418293964 0.8596937380729212 0.10440545236476952
0.775035290773975 0.6609270882693918
9 0.7856604792691282 0.858916517038271 0.09939802573350126
0.7694781590974652 0.6688220144482875
10 0.7768796901009811 0

...

trans1x-128_peptide
0 0.5966027726482804 0.7195728159349181 0.03227014322019266
0.8179721667918127 0.3454234548576871
1 0.595199000271664 0.706013112551893 0.03195419782844333
0.8287454183065226 0.3249996022905073
2 0.5941754050499088 0.7169219499648166 0.035170459936458114
0.8146960144711323 0.3416264030395506
3 0.5944602443530491 0.7194308394162182 0.03527164113484255
0.8068300581264747 0.3531412965106735
4 0.5953972394151401 0.7208716886621906 0.03376567580580804
0.8137834744465051 0.335457537735632
5 0.5992482408110946 0.7265244172634493 0.03148920509533335
0.810442540739283 0.35689158841628055
6 0.5940719083156549 0.7184141805482833 0.03295034681852812
0.8087541433505144 0.3416322321019408
7 0.5980428042401201 0.7157531495587661 0.036275760443384514
0.8298919722356387 0.32832151035484736
8 0.6058090195074087 0.7346388318103605 0.035231961250664134
0.8103231831554243 0.317074867083711
9 0.593021432622584 0.7186661826796016 0.035144075175675506
0.7892825504654313 0.3833824256149371


C:\Users\s_renaud\Documents\GitHub\MSCSAM_TBD\main_model\transvae\trans_models.py:54: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.params['CHAR_WEIGHTS'] = torch.tensor(self.params['CHAR_WEIGHTS'], dtype=torch.float)


0 0.6541345685872626 0.7549236365678704 0.052205489005400785
0.5856543675533123 0.7029362566947683
1 0.655991009110788 0.7586242595381368 0.05862326849702803
0.5831973617128319 0.6770707491105883
2 0.6573441672424687 0.7609925085057806 0.05548732589854551
0.5981403350526455 0.6942720397319719
3 0.6552051530225654 0.7608196232030452 0.05295081572074026
0.6055794372226464 0.6663611732071139
4 0.6510348861013936 0.7543985991190512 0.053950545617783924
0.5821297605523451 0.6910168438755682
5 0.6600175149130149 0.7668079502927658 0.05830067549212349
0.5915375103654387 0.6935467535866
6 0.6536075836659758 0.7587660997558593 0.05411278203996566
0.5948609952807199 0.6835871566330081
7 0.6446656014918064 0.7522145911793493 0.05157239778607759
0.5907502322311464 0.6759952032200476
8 0.6571865091921288 0.7661963371381338 0.05689522206640763
0.6034923808707724 0.6815690488428783
9 0.6534141304083653 0.7583362062546692 0.052777670148148936
0.5954506326083666 0.6691297087514501
10 0.6542930409023281

C:\Users\s_renaud\Documents\GitHub\MSCSAM_TBD\main_model\transvae\trans_models.py:54: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.params['CHAR_WEIGHTS'] = torch.tensor(self.params['CHAR_WEIGHTS'], dtype=torch.float)


0 0.6551342767368535 0.7538667427934514 0.051001679096075454
0.6164872088455731 0.679046885290608
1 0.6513444880854253 0.7441023045786026 0.05074253286636861
0.6094543703660666 0.6565831776498532
2 0.6588813726872794 0.7596153574321425 0.05647510352723941
0.634184205358012 0.6382421336072017
3 0.6523129120774876 0.7535303754945317 0.05192417004008141
0.617148950787217 0.68201425012028
4 0.6552013762458554 0.7480821612852226 0.05412370909161562
0.6175011555032831 0.670180726842782
5 0.6614653540057287 0.7606099481164204 0.05260256326660832
0.6209067886275036 0.6623382936235109
6 0.654968010770354 0.7486652994597671 0.05403519228837738
0.6233097897246873 0.6665387018964359
7 0.6627364635969695 0.7548471317827695 0.05323885083039715
0.6243743573333378 0.6361219546710919
8 0.6585002824740247 0.7572639623825357 0.05781606434228974
0.6268826124979592 0.6649781958527057
9 0.6439538284902281 0.7408303633831403 0.05006756870296173
0.6162696280012866 0.6504025692866411
10 0.6511034693420927 0.74

<H3> This cell concatenates missing saved_info information (usually not necessary when ran in order)

In [1]:
import glob
import pandas as pd

analyses_list = glob.glob("model_analyses\\train//**/*o.csv", recursive=True) #grab all analyses
old_analyses_list = glob.glob("model_analyses\\old\\train//**/*o.csv", recursive=True)

for csv,old_csv in zip(analyses_list,old_analyses_list):
    print(csv)
    analysis = pd.read_csv(csv)
    old_analysis = pd.read_csv(old_csv)
    old_analysis = old_analysis.drop(columns=old_analysis.loc[:,'position_accs':'pca_func_silhouette[3,4]'].columns)
    new_analysis = pd.concat([analysis,old_analysis], axis=1)
    new_analysis.to_csv(csv,index=False)


model_analyses\train\aae-128_peptide_latent128_train\saved_info.csv
model_analyses\train\aae-128_peptide_latent32_train\saved_info.csv
model_analyses\train\aae-128_peptide_latent64_train\saved_info.csv
model_analyses\train\rnn-128_peptide_latent128_train\saved_info.csv
model_analyses\train\rnn-128_peptide_latent32_train\saved_info.csv
model_analyses\train\rnn-128_peptide_latent64_train\saved_info.csv
model_analyses\train\rnnattn-128_peptide_latent128_train\saved_info.csv
model_analyses\train\rnnattn-128_peptide_latent32_train\saved_info.csv
model_analyses\train\rnnattn-128_peptide_latent64_train\saved_info.csv
model_analyses\train\trans1x-128_peptide_latent128_train\saved_info.csv
model_analyses\train\trans1x-128_peptide_latent32_train\saved_info.csv
model_analyses\train\trans1x-128_peptide_latent64_train\saved_info.csv
model_analyses\train\wae-128_peptide_latent128_train\saved_info.csv
model_analyses\train\wae-128_peptide_latent32_train\saved_info.csv
model_analyses\train\wae-128_pept

<H3> This cell runs the python peptides package and finds physicochemical properties of peptide sequences

In [2]:
import peptides
dict_list=[]
for seq in data:
    pep = peptides.Peptide(seq[0])
    dict_list.append(
        {"aliphatic_index":pep.aliphatic_index(),
     "boman":pep.boman(),
     "charge_ph3":pep.charge(pH=3),
     "charge_ph7":pep.charge(pH=7),
     "charge_ph9":pep.charge(pH=11),
    "hydrophobic_moment":pep.hydrophobic_moment(),
    "hydrophobicity":pep.hydrophobicity(),
    "instability_index":pep.instability_index(),
    "isoelectric_point":pep.isoelectric_point(),
    "molecular_weight":pep.molecular_weight()} )